# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [ ]:
## The following libraries might be useful
# !pip install -q langchain-openai
# !pip install -U -q langchain-community
# !pip install -U -q langchain-chroma
# !pip install -U -q datasets
# !pip install -U -q ragas
# !pip install -U -q rouge_score

In [3]:
# Import essential libraries
from pathlib import Path
import pandas as pd
import numpy as np
from operator import itemgetter
import json
import tiktoken
import chromadb
import openai
import llama_index
import os
import re
import logging
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

True

#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [124]:
# Load the files as documents
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir='rag_legal/corpus', recursive=True)
documents = reader.load_data()
print(f"Loaded {len(documents)} docs")

Loaded 698 docs


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [ ]:
# Clean and preprocess the data
from llama_index.core import Document

# Setup logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

processed_docs = []

for i, doc in enumerate(documents):
    text = doc.text if hasattr(doc, 'text') else str(doc)
    
    # Handle missing or corrupted data
    if not text or len(text.strip()) < 10:
        logger.warning(f"Skipping corrupted document: {doc.metadata.get('file_path', f'doc_{i}')}")
        continue
    
    # Add spaces after punctuation if missing
    text = re.sub(r'\.([A-Za-z])', r'. \1', text)
    text = re.sub(r',([A-Za-z])', r', \1', text)
    text = re.sub(r';([A-Za-z])', r'; \1', text)
    text = re.sub(r':([A-Za-z])', r': \1', text)
    
    # Fix words stuck together
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    
    # Replace sensitive info with placeholders
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
    text = re.sub(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', '[PHONE]', text)
    text = re.sub(r'\(\d{3}\)\s?\d{3}[-.\s]?\d{4}', '[PHONE]', text)
    text = re.sub(r'http[s]?://[^\s]+', '[URL]', text)
    text = re.sub(r'www\.[^\s]+', '[URL]', text)
    
    # Remove extra whitespace only
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)  # Preserve paragraph breaks
    text = text.strip()
    
    # Update document
    processed_doc = Document(
        text=text,
        metadata=doc.metadata
    )
    processed_docs.append(processed_doc)



In [126]:
processed_docs[0].text

"MUTUAL NON-DISCLOSURE AGREEMENT Between AND Subject Matter: Effective Date of Agreement: Period , 2017 for Exchange of Information: , 2017 to Period of Confidentiality: THIS AGREEMENT is made as of the Effective Date of Agreement noted above, by and between the above parties. BACKGROUND: I. The parties desire to have discussions of or relating to the Subject Matter for the purposes of evaluating a possible business relationship between them (“Purpose”). The parties may extend the Subject Matter or add additional parties by executing one or more addenda to this Agreement. II. Such discussions may involve disclosure by one party to the other party of confidential, proprietary or trade secret information of its own or its licensors (“Confidential Information” as defined below), during the Period for Exchange of Information. III. Both parties recognize the value of the Confidential Information and that it is in their mutual best interests to maintain the confidential, proprietary and secr

### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [127]:
# Calculate the average, maximum and minimum document length.
doc_lengths = [len(doc.text) for doc in processed_docs]
avg_length = sum(doc_lengths) / len(doc_lengths)
max_length = max(doc_lengths)
min_length = min(doc_lengths)

# Print the average, maximum and minimum document lengths.
print(f"Average document length: {avg_length}")
print(f"Maximum document length: {max_length}")
print(f"Minimum document length: {min_length}")


Average document length: 102689.65329512894
Maximum document length: 986270
Minimum document length: 1445


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [130]:
# Find frequency of occurence of words
from collections import Counter

stop_words = set(stopwords.words('english'))

all_text = []
for doc in processed_docs:
    words = doc.text.split()
    words_no_stop = [w for w in words if w.lower() not in stop_words and len(w) >= 3]
    all_text.extend(words_no_stop)

word_counts = Counter(all_text)
top_20 = word_counts.most_common(20)
least_20 = word_counts.most_common()[-20:]

print(f'20 most common words in the text: {top_20}')
print(f'20 least common words in the text: {least_20}')

20 most common words in the text: [('Company', 124841), ('shall', 104034), ('Section', 70562), ('Agreement', 51336), ('Parent', 46842), ('respect', 27953), ('may', 26134), ('Merger', 25917), ('applicable', 22546), ('Subsidiaries', 22492), ('date', 22320), ('material', 22255), ('prior', 21917), ('Party', 21470), ('set', 20629), ('pursuant', 20612), ('forth', 20235), ('(i)', 19592), ('required', 19060), ('Agreement,', 18847)]
20 least common words in the text: [('tapresearch.', 1), ('com/user/privacy', 1), ('Vungle', 1), ('subpoena);', 1), ('Children.', 1), ('check-ins)', 1), ('info/choices', 1), ('youronlinechoices.', 1), ('eu/', 1), ('non-marketing', 1), ('socket', 1), ('(SSL).', 1), ('22575-22579)', 1), ('(DNT)', 1), ('newer', 1), ('tracked,', 1), ('receive;', 1), ('1651', 1), ('Collins,', 1), ('75080', 1)]


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [131]:
# Transform the page contents of documents
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Transform documents to TF-IDF vectors
texts = [doc.text for doc in processed_docs]
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(texts)

# Calculate similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

# First 10 documents
first_10_sim = similarity_matrix[:10, :10]
print(np.round(first_10_sim, 3))

[[1.    0.852 0.91  0.853 0.872 0.885 0.937 0.735 0.893 0.831]
 [0.852 1.    0.87  0.85  0.867 0.907 0.879 0.725 0.902 0.858]
 [0.91  0.87  1.    0.916 0.918 0.938 0.955 0.715 0.934 0.821]
 [0.853 0.85  0.916 1.    0.912 0.914 0.916 0.677 0.925 0.791]
 [0.872 0.867 0.918 0.912 1.    0.914 0.904 0.684 0.959 0.788]
 [0.885 0.907 0.938 0.914 0.914 1.    0.928 0.713 0.936 0.856]
 [0.937 0.879 0.955 0.916 0.904 0.928 1.    0.707 0.922 0.819]
 [0.735 0.725 0.715 0.677 0.684 0.713 0.707 1.    0.714 0.712]
 [0.893 0.902 0.934 0.925 0.959 0.936 0.922 0.714 1.    0.828]
 [0.831 0.858 0.821 0.791 0.788 0.856 0.819 0.712 0.828 1.   ]]


In [132]:
# create a list of 10 random integers
random_indices = random.sample(range(len(texts)), 10)


In [133]:
# Compute similarity scores for 10 random documents
random_sim = similarity_matrix[random_indices][:, random_indices]
print(np.round(random_sim, 3))

[[1.    0.748 0.874 0.907 0.865 0.719 0.823 0.814 0.931 0.882]
 [0.748 1.    0.737 0.711 0.75  0.592 0.702 0.694 0.79  0.794]
 [0.874 0.737 1.    0.849 0.853 0.682 0.788 0.783 0.895 0.868]
 [0.907 0.711 0.849 1.    0.83  0.676 0.81  0.803 0.919 0.867]
 [0.865 0.75  0.853 0.83  1.    0.701 0.807 0.804 0.899 0.875]
 [0.719 0.592 0.682 0.676 0.701 1.    0.666 0.668 0.711 0.702]
 [0.823 0.702 0.788 0.81  0.807 0.666 1.    0.932 0.833 0.819]
 [0.814 0.694 0.783 0.803 0.804 0.668 0.932 1.    0.824 0.812]
 [0.931 0.79  0.895 0.919 0.899 0.711 0.833 0.824 1.    0.915]
 [0.882 0.794 0.868 0.867 0.875 0.702 0.819 0.812 0.915 1.   ]]


### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [134]:
# Process files and generate chunks
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=128)
chunks = parser.get_nodes_from_documents(processed_docs)

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [ ]:
# Fetch API KEY
import config
from config import openai_api_key
from llama_index.core import Settings
openai_api_key = 'API-KEY'
os.environ['OPENAI_API_KEY'] = openai_api_key
from llama_index.embeddings.openai import OpenAIEmbedding

In [155]:
# Initialise an embedding function
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [ ]:
# Add Chunks to vector DB
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create a directory for vector database
vector_db_dir = "./vector_database"
os.makedirs(vector_db_dir, exist_ok=True)

# Initialize Chroma vector database
chroma_client = chromadb.PersistentClient(path=vector_db_dir)
chroma_collection = chroma_client.create_collection(
    name="chroma_collection_new",
    metadata={"dimension": 1536})

# Create vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the embeddings to vector database
index = VectorStoreIndex(chunks, storage_context=storage_context)

In [158]:
# Verify the database
collection_count = chroma_collection.count()
print(f"Total vectors in database: {collection_count}")

Total vectors in database: 53847


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [ ]:
# Create a RAG chain
from llama_index.llms.openai import OpenAI
custom_llm = OpenAI(model="gpt-4-turbo", temperature=0)
from llama_index.core import PromptTemplate
custom_qa_prompt = PromptTemplate(
    "Context: {context_str}\n"
    "Question: {query_str}\n"
    "Answer the question directly, focusing on ownership and rights granted or not granted:"
)

query_engine = index.as_query_engine(
    similarity_top_k=10,  # Get nearest 10 chunks
    llm=custom_llm,
    response_mode="compact",  # Better for legal text
    text_qa_template=custom_qa_prompt,
    verbose=True  # Display the reasoning steps
)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [197]:
# Create a function for question answering
def ask_question(question):
    response = query_engine.query(question)
    print(f"Q: {question}")
    print(f"A: {response}")
    print(f"Sources: {[node.metadata.get('file_path', 'unknown').split('/')[-1] for node in response.source_nodes]}")
    return response


In [198]:
# Example question
test_question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"


In [250]:
response = ask_question(test_question)

2025-08-26 16:56:16,823 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-26 16:56:20,251 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Q: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
A: Yes, the Non-Disclosure Agreement between CopAcc and ToP Mentors indicates that the Agreement does not grant the Receiving Party any rights to the Confidential Information. The document specifies that all Confidential Information remains the property of the Disclosing Party, and no license, right, interest, or title in or to the Confidential Information is conferred to the Receiving Party other than as expressly set forth in the Agreement.
Sources: ['tpi-non-disclosure-agreement_1.txt', 'NCDG_Non-disclosure-agreement.txt', 'IPTK-CO-MutualNon-DisclosureAgreement.txt', 'IPTK-CO-MutualNon-DisclosureAgreement.txt', 'tpi-non-disclosure-agreement_1.txt', 'thoughtbot-mutual-nda.txt', 'IPTK-CO-MutualNon-DisclosureAgreement.txt', 'IPTK-CO-MutualNon-DisclosureAgreement.txt', 'ICORECONNECTINC_10_13_

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [185]:
# Create a question set by taking all the questions from the benchmark data
reader_bm = SimpleDirectoryReader(input_dir='rag_legal/benchmarks', recursive=True)
benchmarks = reader_bm.load_data()

test_questions = []
ground_truth_answers = []

# Also create a ground truth/answer set
data = json.loads(benchmarks[0].text)
for test in data['tests']:
    test_questions.append(test['query'])
    answers = [s['answer'] for s in test['snippets']]
    ground_truth_answers.append(" ".join(answers))


In [186]:
benchmarks[0].text

'{\n    "tests": [\n        {\n            "query": "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?",\n            "snippets": [\n                {\n                    "file_path": "contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt",\n                    "span": [\n                        11461,\n                        11963\n                    ],\n                    "answer": "Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection 

In [187]:
data

{'tests': [{'query': 'Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?',
   'snippets': [{'file_path': 'contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt',
     'span': [11461, 11963],
     'answer': 'Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.'}]},
  {'query': 'Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document state that Confidential Information shall 

In [251]:
# Check ground truth answer for the first question
print(f"Question: {test_question}")
print(f"Ground truth answer: {ground_truth_answers[0]}")
print(f"\nGenerated answer: {response}")

Question: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Ground truth answer: Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.

Generated answer: Yes, the Non-Disclosure Agreement between CopAcc and ToP Mentors indicates that the Agreement does not grant the Receiving Party any rights to the Confidential Information. The document specifies that all Confidential Information remains the property o

#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [241]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from datasets import Dataset

def evaluate_rag(question, generated_answer, reference_answer, context=None):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(reference_answer, generated_answer)
    
    ref_tokens = reference_answer.lower().split()
    gen_tokens = generated_answer.lower().split()
    bleu = sentence_bleu([ref_tokens], gen_tokens)
    
    results = {
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure,
        'bleu': bleu
    }
    
    if context:
        if not isinstance(context, list):
            context = [context]
            
        dataset = Dataset.from_dict({
            'question': [question],
            'answer': [generated_answer],
            'contexts': [context],  
            'ground_truth': [reference_answer]
        })
        ragas_result = evaluate(
            dataset, 
            metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
        )
        results['faithfulness'] = ragas_result['faithfulness']
        results['answer_relevancy'] = ragas_result['answer_relevancy']
        results['context_precision'] = ragas_result['context_precision']
        results['context_recall'] = ragas_result['context_recall']
    
    return results

def evaluate_rag_batch(questions, generated_answers, reference_answers, contexts=None):
    all_scores = []
    for i in range(len(questions)):
        context = contexts[i] if contexts else None
        
        score = evaluate_rag(
            questions[i],
            generated_answers[i], 
            reference_answers[i],
            context
        )
        all_scores.append(score)
    return all_scores

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [ ]:
# Evaluate the RAG pipeline
generated_answers = []
all_contexts = []
for question in test_questions[:100]:
    response = query_engine.query(question)
    generated_answers.append(str(response))
    contexts = [node.get_content() for node in response.source_nodes]
    all_contexts.append(contexts)

In [ ]:
scores = evaluate_rag_batch(test_questions[:100], generated_answers, ground_truth_answers[:100], contexts=all_contexts)

In [246]:
df_scores = pd.DataFrame({
    'question': test_questions[:100],
    'generated_answer': generated_answers[:100],
    'ground_truth': ground_truth_answers[:100],
    'context': [c[0] if isinstance(c, list) else c for c in all_contexts[:100]],  # Lấy context đầu tiên nếu là list
    'rouge1': [s['rouge1'] for s in scores],
    'rouge2': [s['rouge2'] for s in scores],
    'rougeL': [s['rougeL'] for s in scores],
    'bleu': [s['bleu'] for s in scores],
    'faithfulness': [s.get('faithfulness', None) for s in scores],
    'answer_relevancy': [s.get('answer_relevancy', None) for s in scores],
    'context_precision': [s.get('context_precision', None) for s in scores],
    'context_recall': [s.get('context_recall', None) for s in scores]})
df_scores

question  \
0   Consider the Non-Disclosure Agreement between ...   
1   Consider the Non-Disclosure Agreement between ...   
2   Consider the Non-Disclosure Agreement between ...   
3   Consider the Non-Disclosure Agreement between ...   
4   Consider the Non-Disclosure Agreement between ...   
..                                                ...   
95  Consider FNHA's Non-Disclosure Agreement; Does...   
96  Consider FNHA's Non-Disclosure Agreement; Does...   
97  Consider FNHA's Non-Disclosure Agreement; Does...   
98  Consider FNHA's Non-Disclosure Agreement; Does...   
99  Consider FNHA's Non-Disclosure Agreement; Does...   

                                     generated_answer  \
0   Yes, the Non-Disclosure Agreement between CopA...   
1   No, the Non-Disclosure Agreement between CopAc...   
2   Yes, the Non-Disclosure Agreement between CopA...   
3   No, the Non-Disclosure Agreement between CopAc...   
4   No, the Non-Disclosure Agreement between CopAc...   
..                                                ...   
95  Yes, FNHA's Non-Disclosure Agreement does allo...   
96  No, the FNHA's Non-Disclosure Agreement does n...   
97  No, the FNHA's Non-Disclosure Agreement does n...   
98  Yes, the FNHA Non-Disclosure Agreement allows ...   
99  Yes, the FNHA's Non-Disclosure Agreement restr...   

                                         ground_truth  \
0   Any and all proprietary rights, including but ...   
1   “Confidential Information” means any Idea disc...   
2   Notwithstanding the termination of this Agreem...   
3   At Organiser’s first request, Mentor shall: (d...   
4   Mentor shall not disclose any Confidential Inf...   
..                                                ...   
95  1. Definition. "Confidential Information" mean...   
96  2. Non-Disclosure. Company will not make copie...   
97  Company will not make copies of, disclose, dis...   
98  This Agreement does not preclude discussions o...   
99  Company will not use the Confidential Informat...   

                                              context    rouge1    rouge2  \
0   The Receiving Party understands that nothing h...  0.391608  0.099291   
1   1. Definition of Confidential Information Conf...  0.178571  0.037037   
2   7 Notice of Required Disclosure If Mentor is r...  0.649573  0.643478   
3   Notwithstanding Mentor’s right to assess and r...  0.421053  0.106667   
4   The coaching service provided by Mentor is wor...  0.400000  0.204082   
..                                                ...       ...       ...   
95  2. Non-Disclosure. Company will not make copie...  0.382979  0.100719   
96  2. Non-Disclosure. Company will not make copie...  0.298246  0.196429   
97  2. Non-Disclosure. Company will not make copie...  0.409091  0.372093   
98  2. Non-Disclosure. Company will not make copie...  0.279570  0.065934   
99  2. Non-Disclosure. Company will not make copie...  0.326087  0.288889   

      rougeL           bleu          faithfulness      answer_relevancy  \
0   0.265734   1.549324e-78                [0.75]  [0.9535719153673018]   
1   0.107143  4.497378e-232                 [0.5]  [0.9887708840650459]   
2   0.649573   4.769512e-01                 [1.0]  [0.9779471212542884]   
3   0.210526   1.762805e-78                 [0.5]    [0.93369961185781]   
4   0.300000   1.184471e-01                 [1.0]  [0.9154413184140688]   
..       ...            ...                   ...                   ...   
95  0.226950  3.936423e-155                 [1.0]  [0.9801964709676256]   
96  0.280702   5.689226e-02                 [1.0]  [0.9530030853957898]   
97  0.409091   1.809275e-01  [0.8571428571428571]  [0.9737618473900701]   
98  0.150538  4.143286e-155                 [1.0]   [0.956094384467065]   
99  0.326087   1.608382e-01                 [1.0]  [0.9563383558587946]   

       context_precision        context_recall  
0   [0.6555555555337037]  [0.6666666666666666]  
1                  [0.0]                 [1.0]  
2   [0.5277777777

In [ ]:
# Calculate the average scores
def extract_value(x):
    if isinstance(x, list):
        return x[0] if len(x) > 0 else None
    return x
# Apply cho tất cả RAGAS columns
ragas_cols = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
for col in ragas_cols:
    if col in df_scores.columns:
        df_scores[col] = df_scores[col].apply(extract_value)

average_scores = df_scores[['rouge1', 'rouge2', 'rougeL', 'bleu', 
                         'faithfulness', 'answer_relevancy', 
                         'context_precision', 'context_recall']].mean()
print(average_scores)

rouge1               0.468946
rouge2               0.277599
rougeL               0.364876
bleu                 0.169314
faithfulness         0.839546
answer_relevancy     0.949508
context_precision    0.764367
context_recall       0.959167
dtype: float64


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

## Data Insights
The analysis of legal documents reveals a highly specialized corpus with significant document length variation (1,445 to 986,270 characters, averaging ~103K), dominated by merger agreements and contracts as evidenced by the most frequent terms. The high similarity scores between documents (0.67-0.96) indicate substantial structural and terminological consistency across the legal dataset, which benefits retrieval accuracy.


## Data & RAG pipeline

The corpus exhibits strong domain-specific language patterns with legal terminology ("shall," "pursuant," "Agreement") appearing consistently throughout
Document similarity analysis shows clustering patterns that could be leveraged for better categorization and retrieval
The chunking strategy (512 tokens with 128 overlap) appears well-suited for capturing legal clauses while maintaining context
The text-embedding-3-small model with 1,536 dimensions provides adequate semantic representation for legal concepts

## RAG Performance Analysis
The evaluation metrics demonstrate strong retrieval capabilities with high faithfulness (0.84) and answer relevancy (0.95), indicating the system effectively identifies and uses relevant legal passages. However, the moderate ROUGE scores (0.47 for ROUGE-1) and low BLEU score (0.17) suggest opportunities for improvement in generation quality.